In [ ]:
login_url ="https://www.linkedin.com/jobs/search/?currentJobId=0&f_E=1&f_JT=F%2CI&f_T=13&f_TPR=r86400&f_WT=1%2C3&geoId=101282230&keywords=data%20science%20praktikum&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R"

In [239]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import ollama
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import re
import random
import json
import ollama
import os
from dotenv import load_dotenv
import smtplib
from email.message import EmailMessage
import webbrowser

In [240]:
def startEngine():
    load_dotenv()
    mail =  os.getenv("mail_1")
    password = os.getenv("password_1")
    driver = webdriver.Chrome()
    driver.get(login_url)
    time.sleep(2)
    login_button = driver.find_element("xpath", '//*[@id="base-contextual-sign-in-modal"]/div/section/div/div/div/div[2]/button')
    login_button.click()
    time.sleep(2)
    e_mail = driver.find_element("xpath", '//*[@id="base-sign-in-modal_session_key"]')
    e_mail.send_keys("leelynd.boede@minuteafter.com")
    password_input = driver.find_element("xpath", '//*[@id="base-sign-in-modal_session_password"]')
    password_input.send_keys("dybtax-8cyTha-bofmer")

    button_2 = driver.find_element("xpath", '//*[@id="base-sign-in-modal"]/div/section/div/div/form/div[2]/button')
    button_2.click()
    time.sleep(10)



    location_button = driver.find_element(By.CSS_SELECTOR, "input[aria-label='Ort, Bundesland/Kanton oder PLZ']")
    location_element = driver.find_element(By.CSS_SELECTOR, "input[data-job-search-box-location-input-trigger]")
    location = location_element.get_attribute("data-job-search-box-location-input-trigger")
    print("Location checking...")
    if location != "Deutschland":
        time.sleep(1)
        location_button.clear()
        location_button.send_keys("Germany")
        suche_button = driver.find_element("xpath", "//*[@id=\"global-nav-search\"]/div/div[2]/button[1]")
        time.sleep(2)
        suche_button.click()
        print("location corrected to germnay")
    else:
        print("Location is already germany")

    return driver

#driver = startEngine()

In [241]:
def get_Click_Directions(driver):
    jobs_all = driver.find_elements(By.CSS_SELECTOR, "li.scaffold-layout__list-item")

    

    return jobs_all

#jobs = get_Click_Directions(driver)


In [242]:
# get the infos

def Job_Content(driver):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    About_The_Job = soup.find('article', class_='jobs-description__container') # all jobs
    Company_Name = soup.find('div', class_='job-details-jobs-unified-top-card__company-name')
    titel_h1 = soup.find('div', class_="job-details-jobs-unified-top-card__job-title")
    Link_to_stelle = titel_h1.find('a')

    return About_The_Job.get_text(strip =True), Company_Name.get_text(strip = True), Link_to_stelle["href"]

#job_description, Company_Name, Location, Link_to_stelle = Job_Content()


In [ ]:
def starting_mail():
    msg = EmailMessage()
    content = "Scraping ist gestartet"
    msg.set_content(content)

    msg["Subject"]= "Scraping starts..."
    msg['From'] = os.getenv(login_mail)
    msg['To'] = os.getenv(mail_to)

    s = smtplib.SMTP("smtp-relay.brevo.com", 587)
    s.starttls()
        mail = os.getenv(mail_mailsender)
    token = os.getenv(pass_mailsender)
    s.login(mail, token) 
    s.send_message(msg)
    s.quit()
    return print("mail on the way")

In [ ]:
def finish_mail(links):
    msg = EmailMessage()
    content = f"{links}"
    msg.set_content(content)

    msg["Subject"]= "all jobs are scraped"
    msg['From'] = "ihsan2001.blm@gmail.com"
    msg['To'] = "ichsanbolum@gmail.com"

    s = smtplib.SMTP("smtp-relay.brevo.com", 587)
    s.starttls()
    mail = os.getenv(mail_mailsender)
    token = os.getenv(pass_mailsender)
    s.login(mail, token) 
    s.send_message(msg)
    s.quit()
    return print("mail on the way")

In [245]:

def Ollama_Rate(job_description):
    prompt = f"""
        Analyze the following job posting for an INTERNSHIP.

        CONTEXT:
        - Job Description: {job_description[:4000]}

        ==================================================
        TASK:
        Determine if this job is suitable for a Bachelor internship based on the rules below.

        ==================================================
        STEP 1 — CRITICAL FACT CHECK
        Check the text for the following constraints. Answer internally with YES/NO.

        1. IS_INTERNSHIP: Does it explicitly offer an Internship/Praktikum? (Reject if only Working Student/Werkstudent).
        2. DURATION_OK: Is the duration at least 5 months (or "3-6 months", "6 months")? (Reject if only 3 months or less).
        3. LOCATION_OK: Is it NOT fully remote? (Accept hybrid/on-site).
        4. LEVEL_OK: Is it suitable for Bachelor students?
        5. FIELD_OK: Is it in a relevant field (e.g., Data Science, Software Engineering)?

        ==================================================
        DECISION LOGIC:
        - If ANY strict negative rule applies (e.g., only Werkstudent, <5 Months) -> REJECT.
        - If specific duration is missing but context implies a substantial internship -> ACCEPT.
        
        ==================================================
        OUTPUT FORMAT:
        Return ONLY a valid JSON object. Do not include markdown formatting (like ```json).

        {{
        "status": "accepted" OR "rejected",
        "reason": "Brief explanation why (max 10 words). E.g., 'Only 3 months duration' or 'Only Werkstudent offered'"
        }}
        """
    ollama_filter = ollama.chat(
        model="qwen3:8b", 
        format="json",
        messages=[{'role': 'user', 'content': prompt}],
        options={"temperature": 0.0}
        )
    chat = ollama_filter["message"]['content']
    response = json.loads(chat)
    return response

In [ ]:
driver = startEngine()

starting_mail()

job_container = []
regex_logic = r"\/jobs\/view\/\d+"
links_holder=[]
with open("scraped_so_far.csv", "r") as seen_jobs:
    stellen_im_text = seen_jobs.read().splitlines()
    print(stellen_im_text)
    scraped_so_far = stellen_im_text
print("Liste",scraped_so_far)
print(len(scraped_so_far))
scraped_jops = []
counter = 0
while True:


    jobs = get_Click_Directions(driver) # where to click: Job Listings path
    print(f"Found {len(jobs)} jobs on this page.")
    
    time.sleep(2)

    for job in jobs:
        y = random.uniform(1.5, 5)
        time.sleep(y)
        job.click()
        print("is being processed", job)
        job_description, Company_Name, Link_to_stelle = Job_Content(driver)
        link_match = re.search(regex_logic, Link_to_stelle)
        find_Link = link_match.group()
        link_creation = f"https://www.linkedin.com{find_Link}"
        print(link_creation)
        with open("scraped_so_far.csv", "a+") as write_into:
            write_into.write(f"{link_creation}\n")

        print("Checking, if the job is already seen")
        if link_creation not in scraped_so_far:
            print("new job")
            links.append(link_creation)
            print("ollama thinks")
            ollama_response = Ollama_Rate(job_description)
            print("ollama said", ollama_response)
            
            if ollama_response.get("status") == "accepted":
                job_container.append([
                    Job_Content(driver)
                    ])

                print("Praktikum Gefunden")    
                print("link",job_container[counter])
                links_holder.append(link_creation)
                #webbrowser.open_new_tab(create_link)
                #cmd = f"open -g {link_creation}"
                #os.system(cmd)
                counter += 1
        else:
            print("job is already seen")

    class_name = 'artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view jobs-search-pagination__button jobs-search-pagination__button--next'.replace(" ", ".")
    print(f"No more job listing... going to next page ")
    try:
        next_page_button = driver.find_element(By.CLASS_NAME, f'{class_name}')
        next_page_button.click()
        print("Clicing on the next page")
    except Exception as e:
        print(f"No more pages to click. Exiting. {e}")

        finish_mail(links_holder)

        print("Mail sent")
        driver.quit()
        break




In [ ]:
filename = "scraped_so_far.csv"

# Opening in 'w' mode automatically wipes the file
with open(filename, "w") as file:
    pass